In [1]:
import numpy as np
import pandas as pd
import h5py
import random
import os
import pyedflib
from tqdm import tqdm
from glob import glob

In [2]:
from data.data_utils import *

# Test split patients

In [12]:
filepath = TRAIN_DATA_DIR

In [19]:
from pathlib import Path

patients = []
entries = Path(filepath)
for view in entries.iterdir():
    for unique_id in view.iterdir():
        for pat_id in unique_id.iterdir():
            patients.append(pat_id.name)

In [22]:
patients = list(set(patients))

print(patients)
print(len(patients))

['00000287', '00001820', '00000418', '00010023', '00001843', '00000529', '00005452', '00000302', '00010489', '00002657', '00009162', '00002707', '00005275', '00005265', '00005427', '00000272', '00002448', '00007252', '00001620', '00008552', '00008480', '00010164', '00002847', '00003419', '00001204', '00008738', '00006351', '00002807', '00010158', '00001545', '00005371', '00008092', '00002435', '00010020', '00008303', '00008476', '00000569', '00006230', '00009030', '00010291', '00010079', '00009669', '00008561', '00003674', '00001357', '00001795', '00003053', '00001587', '00004569', '00008953', '00010798', '00002236', '00003977', '00008018', '00002806', '00009880', '00004892', '00010427', '00010587', '00005672', '00000143', '00005347', '00008615', '00002991', '00007038', '00008053', '00000775', '00001413', '00002796', '00009885', '00006251', '00005512', '00007170', '00006904', '00008829', '00000850', '00008481', '00004473', '00003489', '00006413', '00001169', '00006501', '00000244', '00

In [23]:
file_name = '/share/pi/rubin/eeg/TUH/v1_4_0/train_dev_test/dev_test/01_tcp_ar/098/00009842/s001_2013_04_01/00009842_s001_t000.edf'

patient = file_name[-22:-14]
print(patient)

00009842


['/']


In [6]:
unique_id_folders = [x for x in os.path.join(curr_path, )]

0it [00:00, ?it/s]


# Test resize first then normalize

In [8]:
file_dir = TRAIN_RAW_SEIZURE_FILE
TRAIN_INCREASES = len(Fs) * len(Ws) * len(Os)

train_file_tuples = parseTxtFiles(file_dir,  num_folds=None, fold_idx=None, cross_val=False)[0] 

# Compute features for train set
features = {}
        
for idx in range(1):
    curr_file_name, seizure_class, seizure_idx = train_file_tuples[idx]
    print(curr_file_name)
    # read file
    f = pyedflib.EdfReader(curr_file_name)
        
    ordered_channels = getOrderedChannels(curr_file_name, False, f.getSignalLabels())
       
    signals = getEDFsignals(f)
                
    frequencies = getSamplingFreq(f, ordered_channels)
    freq = frequencies[0]        
            
    seizure_times = getSeizureTimes(curr_file_name, file_type="edf")
                
    f._close()
        
                
    # dense features, only for training split
    dense_feats = []
    for sampling_idx in range(TRAIN_INCREASES):
        dense_param = DENSE_PARAMS[sampling_idx]
        dense_feats.append(denseSampling(ordered_channels, signals, dense_param[0], dense_param[1], dense_param[2]))
                
print(dense_feats)

/share/pi/rubin/eeg/TUH/v1_4_0/train_dev_test/train/03_tcp_ar_a/065/00006514/s010_2010_05_04/00006514_s010_t005.edf
[array([[[ 99.52321736, 193.9868442 ,  99.35165872],
        [ 99.52306937, 126.19272012, 100.87990795],
        [ 99.52283295,  74.29263947,  98.08844001],
        ...,
        [ 99.52270076,  86.50905172, 100.35202514],
        [ 99.52322337, 113.70928784,  98.86892945],
        [ 99.52312105, 225.        , 100.09805313]],

       [[ 99.52321736, 193.9868442 ,  99.35165872],
        [ 99.52306937, 126.19272012, 100.87990795],
        [ 99.52283295,  74.29263947,  98.08844001],
        ...,
        [ 99.52270076,  86.50905172, 100.35202514],
        [ 99.52322337, 113.70928784,  98.86892945],
        [ 99.52312105, 225.        , 100.09805313]],

       [[ 99.52321736, 193.9868442 ,  99.35165872],
        [ 99.52306937, 126.19272012, 100.87990795],
        [ 99.52283295,  74.29263947,  98.08844001],
        ...,
        [ 99.52270076,  86.50905172, 100.35202514],
        

# Test reading seizure class

In [7]:
eg_file = '/share/pi/rubin/eeg/TUH/v1_4_0/train_dev_test/train/02_tcp_le/060/00006083/s003_2010_10_11/00006083_s003_t001.edf'

seizure_class = getSeizureClass(eg_file, file_type='edf')
print(seizure_class)

seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
seizure str:['cpsz']
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [8]:
tse_file = eg_file[:-4] + ".tse"
        
with open(tse_file) as f:
    for line in f.readlines():
        print(line)

version = tse_v1.0.0



0.0000 57.8680 bckg 1.0000

57.8680 162.8680 cpsz 1.0000

162.8680 416.8000 bckg 1.0000

416.8000 467.9000 cpsz 1.0000

467.9000 723.5560 bckg 1.0000

723.5560 742.1560 cpsz 1.0000

742.1560 776.6560 bckg 1.0000

776.6560 788.1560 cpsz 1.0000

788.1560 947.1120 bckg 1.0000

947.1120 967.8240 cpsz 1.0000

967.8240 996.1120 bckg 1.0000

996.1120 1014.1120 cpsz 1.0000

1014.1120 1021.1120 bckg 1.0000

1021.1120 1051.1120 cpsz 1.0000

1051.1120 1071.0320 bckg 1.0000

1071.0320 1083.9880 cpsz 1.0000

1083.9880 1099.1320 bckg 1.0000

1099.1320 1118.1320 cpsz 1.0000

1118.1320 1128.1320 bckg 1.0000

1128.1320 1144.1320 cpsz 1.0000

1144.1320 1156.0880 bckg 1.0000

1156.0880 1176.0880 cpsz 1.0000

1176.0880 1187.0880 bckg 1.0000

1187.0880 1208.1880 cpsz 1.0000

1208.1880 1247.1120 bckg 1.0000

1247.1120 1278.0320 cpsz 1.0000

1278.0320 1372.0880 bckg 1.0000

1372.0880 1388.0880 cpsz 1.0000

1388.0880 1403.0880 bckg 1.0000

1403.0880 1424.0880 cpsz 1.0000

1424.0880 143

# Test reading EDF file

In [4]:
file_name = '/share/pi/rubin/eeg/TUH/v1_4_0/train_dev_test/train/02_tcp_le/071/00007128/s001_2010_10_12/00007128_s001_t001.edf'

f = pyedflib.EdfReader(file_name)

In [8]:
channel = 1
freq = f.getSampleFrequency(channel)
freq

250

In [9]:
INCLUDED_CHANNELS = ['EEG FP1', 'EEG FP2', 'EEG F3', 'EEG F4', 'EEG C3', 'EEG C4', 'EEG P3', 'EEG P4',
'EEG O1', 'EEG O2', 'EEG F7', 'EEG F8', 'EEG T3', 'EEG T4', 'EEG T5', 'EEG T6', 'EEG FZ', 'EEG CZ', 'EEG PZ']
len(INCLUDED_CHANNELS)

19

# Feature Map S (Fast Fourier Transform)
The FFT of an input signal of N points, will return an vector of N points. The first half of this vector (N/2 points) contain the useful values  of the frequency spectrum from 0 Hz up to the Nyquist frequency of f_s / 2. The second half contains the complex conjugate and can be disregarded since it does not provide any useful information.

In [27]:
from scipy.fftpack import fft
import math

def computeTimeFreqRep(signals, freq = 12, overlap = 0.25, window = 1):
    """
    Args:
        signals: EEG segment, shape (number of channels, number of data points)
        freq: sampling frequency in Hz
        overlap: proportion of overlap with previous segment
        window: window size in seconds
    Returns:
        S: time-frequency representation, shape (number of data points, number of channels)
    """
    n_ch, t_len = signals.shape

    start_time = 0
    end_time = int(freq * window)
    signal_segs = [signals[:,start_time:end_time]]

    while end_time < t_len:        
        offset = int(freq * window * overlap)
        start_time = end_time - offset
        end_time = min(t_len, start_time + int(freq * window))
        curr_seg = signals[:, start_time:end_time]
        if curr_seg.shape[1] < freq * window:
            diff = freq * window - curr_seg.shape[1]
            curr_seg = np.concatenate((curr_seg, np.zeros((n_ch, diff))), axis=1)
        signal_segs.append(curr_seg)
    
    signal_segs = np.concatenate(signal_segs, axis=1)
    N = signal_segs.shape[1]
    fft_values_ = fft(signal_segs)
    S = 1.0/N * np.abs(fft_values_)

    return S

signals = np.random.rand(20,200000)
S = computeTimeFreqRep(signals)
print('Original signals shape:{}'.format(signals.shape))
print('FFT values shape:{}'.format(S.shape))

Original signals shape:(20, 200000)
FFT values shape:(20, 266664)


# Feature Map D (Partial Derivative)

In [28]:
from scipy.fftpack import diff

def computeDivergenceMap(S):
    """
    Args:
        S: time-frequency representation from computeTimeFreqRep, shape (number of data points, number of channels)
    Returns:
        D: divergence map, shape (number of data points, number of channels)
    """
    D_x = np.zeros(S.shape)
    D_y = np.zeros(S.shape)

    for i in range(S.shape[0]):
        D_x[i,:] = diff(S[i,:])

    
    for j in range(S.shape[1]):
        D_y[:,j] = diff(S[:,j])

    D = D_x + D_y
    
    return D

In [29]:
D = computeDivergenceMap(S)
D

array([[ 1.18559431e-03, -2.12081524e+04,  1.05980342e+04, ...,
         7.06019915e+03, -1.05980303e+04,  2.12081554e+04],
       [-3.27149498e-03, -2.11649666e+04,  1.05693792e+04, ...,
         7.08593816e+03, -1.05693834e+04,  2.11649663e+04],
       [-3.18229155e-04, -2.11700566e+04,  1.05832448e+04, ...,
         7.03905323e+03, -1.05832433e+04,  2.11700561e+04],
       ...,
       [ 5.57197659e-03, -2.11490165e+04,  1.05595568e+04, ...,
         7.01961411e+03, -1.05595564e+04,  2.11490216e+04],
       [-2.17832704e-03, -2.11786772e+04,  1.05672785e+04, ...,
         7.04060350e+03, -1.05672773e+04,  2.11786739e+04],
       [ 2.96704010e-03, -2.11590485e+04,  1.05681661e+04, ...,
         7.03520119e+03, -1.05681661e+04,  2.11590468e+04]])

# Feature Map I (Transformation of D)

In [30]:
def computeTransformI(D, smooth_window = 8):
    """
    Args:
        D: divergence map, shape (number of data points, number of channels)
    Returns:
        I: feature map I, shape (number of data points, number of channels)
    """

    box = np.ones((smooth_window, smooth_window)) / smooth_window
    I = convolve2d(D, box, mode='same', boundary='fill', fillvalue=0)
            
    return I

In [31]:
from scipy.signal import convolve2d

I = computeTransformI(D)
I.shape

(20, 266664)

# Concatenate all feature maps

In [32]:
def computeDivSpec(signals, freq = 12, overlap = 0.25, window = 1):
    """
    Args:
        signals: EEG segment, shape (number of channels, number of data points)???
    Returns:
        Ds: DivSpec feature maps, shape (224, 224, 3)
    """
    
    # S
    S = computeTimeFreqRep(signals, freq=freq, overlap=overlap, window=window)
    print(S.shape)
    
    # D
    D = computeDivergenceMap(S)
    print(D.shape)
    
    # I
    I = computeTransformI(D, smooth_window=SMOOTH_W)
    print(I.shape)
    
    Ds = np.stack([S, D, I], axis = 2)
    Ds_norm = 255 * (Ds - np.amin(Ds)) / (np.amax(Ds) - np.amin(Ds))
    
    Ds_res = cv2.resize(Ds_norm, dsize=(224, 224)) # resize to (224, 224, 3)
    
    return Ds_res

In [33]:
from constants import *
import cv2

Ds_res = computeDivSpec(signals)
print(np.amax(Ds_res))
print(np.amin(Ds_res))
print('Shape of Ds_res:{}'.format(Ds_res.shape))

(20, 266664)
(20, 266664)
(20, 266664)
127.88328068806709
127.12404325619593
Shape of Ds_res:(224, 224, 3)


# Fast Fourier Transform 2

In [4]:
import pyedflib
import numpy as np
from data.data_utils import *

In [5]:
file_name = '/share/pi/rubin/eeg/TUH/v1_4_0/train_dev_test/train/02_tcp_le/002/00000281/s001_2003_06_21/00000281_s001_t000.edf'
f = pyedflib.EdfReader(file_name)
ordered_channels = getOrderedChannels(file_name, False, f.getSignalLabels())
signals = getEDFsignals(f)
signals = signals[ordered_channels,:]
freq = f.getSampleFrequency(0)
print(freq)
print('signals shape:{}'.format(signals.shape))
f._close()

250
signals shape:(19, 302250)


In [10]:
from scipy.signal import hilbert
from scipy.signal import welch
from scipy.fftpack import fft
import time

N = signals.shape[1]

# hilbert transform
hilbert_signal = hilbert(signals, axis=1)
#print(np.absolute(hilbert_signal[0,:]))
print('hilbert_signal:{}'.format(hilbert_signal))

# fourier transform
start_t = time.time()
fourier_signal = np.fft.fft(hilbert_signal, axis=1)
et = time.time() - start_t
print('FFT from numpy:{}'.format(fourier_signal))
print('Time from numpy:{}'.format(et))
#print(fourier_signal.shape)

start_t = time.time()
fourier_scipy = fft(hilbert_signal, axis=1)
et = time.time() - start_t
print('Time from scipy:{}'.format(et))
print('FFT from scipy:{}'.format(fourier_scipy))

#S = 1.0/N * np.abs(fourier_signal)
#print('S:{}'.format(S))
#print('S shape:{}'.format(S.shape))

hilbert_signal:[[-24.87183905  -7.53997203j -24.87183905 -19.18272262j
  -24.87183905 -19.62667457j ...  -3.05175939 -16.42969122j
   -5.18799097 -15.69300812j  -7.32422254  -6.06718182j]
 [-24.87183905 -69.80066923j -24.87183905 -66.33157001j
  -24.87183905 -66.46145602j ... -26.85548265 -65.27074635j
  -28.38136235 -64.45138618j -30.51759392 -68.6465514j ]
 [-24.87183905 -91.75965055j -24.87183905 -81.32256795j
  -24.87183905 -81.22961696j ... -45.166039   -77.12794699j
  -43.94533525 -81.39409541j -41.80910367 -92.3718688j ]
 ...
 [-24.87183905-107.52475703j -24.87183905 -94.05541239j
  -24.87183905 -93.60091296j ... -44.25051119 -88.95290345j
  -48.82815027 -90.42384249j -47.91262246-108.23152416j]
 [-24.87183905-168.77913944j -24.87183905-137.2098824j
  -24.87183905-137.20884293j ... -68.66458632-129.68881527j
  -77.20951262-131.18530428j -76.59916074-168.89314532j]
 [-24.87183905-102.846089j   -24.87183905 -91.60506035j
  -24.87183905 -90.7628412j  ... -38.75734428 -85.11084085j


# Check sampling frequencies

In [23]:
from data.data_utils import parseTxtFiles

SEIZURE_FILE = 'data/seizure_files.txt'

file_tuples_list = parseTxtFiles(SEIZURE_FILE, 5, 0, cross_val=True)
file_tuples = file_tuples_list[0] + file_tuples_list[1]

In [24]:
import pyedflib
from data.data_utils import getOrderedChannels, getSamplingFreq

freq_list = [250, 256, 400, 512]
freq_counts = np.zeros(len(freq_list))

for curr_file_name, seizure_class, seizure_idx in file_tuples:
    f = pyedflib.EdfReader(curr_file_name)
    ordered_channels = getOrderedChannels(curr_file_name, False, f.getSignalLabels())
    curr_freq = getSamplingFreq(f, ordered_channels)[0]
    
    freq_idx = freq_list.index(curr_freq)
    freq_counts[freq_idx] += 1
    
    f._close()

In [25]:
freq_counts / sum(freq_counts)

array([0.43006471, 0.41961175, 0.14783474, 0.0024888 ])

In [1]:
sum(freq_counts)

NameError: name 'freq_counts' is not defined

In [2]:
2009 * 0.0024888

4.9999992

# Test data loader

In [1]:
from data.data_loader import SeizureDataset
from data.data_utils import *
from constants import *
import torch.utils.data as data

In [2]:
train_dataset = SeizureDataset(TRAIN_SEIZURE_FILE, num_folds=None, fold_idx=None, cross_val=False, split='train')
print(train_dataset)
train_loader = data.DataLoader(dataset=train_dataset,
                                shuffle=True,
                                batch_size=4,
                                num_workers=8)
print(len(train_loader))

2


In [3]:
for features, y, file_name in train_loader:
    print(features.shape)
    print(y.shape)
    print(file_name)

torch.Size([4, 3, 224, 224])
torch.Size([4, 1])
('/share/pi/rubin/eeg/TUH/v1_4_0/train_dev_test/train/03_tcp_ar_a/071/00007196/s002_2010_01_15/00007196_s002_t001.edf', '/share/pi/rubin/eeg/TUH/v1_4_0/train_dev_test/train/03_tcp_ar_a/071/00007196/s002_2010_01_15/00007196_s002_t001.edf', '/share/pi/rubin/eeg/TUH/v1_4_0/train_dev_test/train/03_tcp_ar_a/071/00007196/s002_2010_01_15/00007196_s002_t001.edf', '/share/pi/rubin/eeg/TUH/v1_4_0/train_dev_test/train/03_tcp_ar_a/071/00007196/s002_2010_01_15/00007196_s002_t001.edf')


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/share/pi/rubin/envs/tsy935/eeg/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2878, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-a7f8104af690>", line 1, in <module>
    for features, y, file_name in train_loader:
  File "/share/pi/rubin/envs/tsy935/eeg/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 576, in __next__
    idx, batch = self._get_batch()
  File "/share/pi/rubin/envs/tsy935/eeg/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 553, in _get_batch
    success, data = self._try_get_batch()
  File "/share/pi/rubin/envs/tsy935/eeg/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 511, in _try_get_batch
    data = self.data_queue.get(timeout=timeout)
  File "/share/pi/rubin/envs/tsy935/eeg/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/share/pi/rubin/envs/tsy935/eeg/lib

KeyboardInterrupt: 

# Test using convolution to compute feature map I

In [4]:
import numpy as np
from scipy import signal

mat = np.random.rand(5, 10)
mat

array([[0.76366077, 0.45397539, 0.95809204, 0.862023  , 0.77290162,
        0.11088182, 0.31168784, 0.70982075, 0.16948897, 0.9344236 ],
       [0.1549295 , 0.92068187, 0.86685409, 0.69315973, 0.22127316,
        0.92222117, 0.35484596, 0.91699444, 0.48796683, 0.69511583],
       [0.85308016, 0.69541773, 0.84233628, 0.53600992, 0.38086963,
        0.9907038 , 0.04810722, 0.22477857, 0.20622166, 0.9809442 ],
       [0.05152033, 0.62272399, 0.01380563, 0.33791412, 0.2783711 ,
        0.99760072, 0.30544162, 0.38173608, 0.46634219, 0.52232156],
       [0.09224233, 0.75910038, 0.35305063, 0.74056764, 0.68197494,
        0.88755354, 0.92321861, 0.11357926, 0.28093891, 0.96503515]])

In [14]:
window = np.ones((3,3)) / 9.
window

array([[0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111]])

In [15]:
I = signal.convolve2d(mat, window, 'same')

In [16]:
I.shape

(5, 10)

In [17]:
I

array([[0.25480528, 0.45757707, 0.52830957, 0.48603374, 0.39805117,
        0.2993124 , 0.36960577, 0.3278672 , 0.43486782, 0.25411058],
       [0.4268606 , 0.72322531, 0.75872778, 0.68150216, 0.61000487,
        0.45705469, 0.51000462, 0.38110136, 0.59175054, 0.3860179 ],
       [0.36648373, 0.55792773, 0.6143226 , 0.46339929, 0.59534704,
        0.49993715, 0.57138106, 0.37693717, 0.54249126, 0.37321247],
       [0.34156499, 0.47591972, 0.54454737, 0.46276665, 0.64795171,
        0.6104268 , 0.54141327, 0.32781824, 0.46021084, 0.38020041],
       [0.16950967, 0.21027148, 0.31412916, 0.26729823, 0.43599801,
        0.4526845 , 0.40101443, 0.27458408, 0.30332813, 0.24829309]])

In [4]:
import torch

In [5]:
A = torch.zeros(3,2)

In [6]:
A = A.view(-1)

In [8]:
A.shape

torch.Size([6])

# Test sliding window using itertools

In [62]:
from itertools import islice
import numpy as np

def getSignalSegs(signals, freq = 12, overlap = 0.25, window = 1, copy = True):
    sh = (signals.size - int(freq * window) + 1, int(freq * window))
    st = signals.strides * 2
    o = int(freq * window * overlap)
    view = np.lib.stride_tricks.as_strided(signals, strides = st, shape = sh)[0::o]
    if copy:
        return view.copy()
    else:
        return view
    
    #signals = tuple(map(tuple, signals.T))
    #signal_segs = []
    #step = int(window * freq * (1 - overlap))
    #start = 0
    
   # it = iter(signals)
   # signal_segs = np.array(list(islice(it, start, None, step)))
            
    #return signal_segs


def window(a, w = 4, o = 2, copy = False):
    sh = (a.size - w + 1, w)
    st = a.strides * 2
    view = np.lib.stride_tricks.as_strided(a, strides = st, shape = sh)[0::o]
    if copy:
        return view.copy()
    else:
        return view


In [63]:
import pyedflib
import numpy as np
from data.data_utils import *

file_name = '/share/pi/rubin/eeg/TUH/v1_4_0/train_dev_test/train/02_tcp_le/002/00000281/s001_2003_06_21/00000281_s001_t000.edf'
f = pyedflib.EdfReader(file_name)
ordered_channels = getOrderedChannels(file_name, False, f.getSignalLabels())
signals = getEDFsignals(f)
signals = signals[ordered_channels,:]
f._close()

print(signals.shape)

(19, 302250)


In [74]:
start_t = time.time()
signal_segs = getSignalSegs(signals[4,:], freq = 12, overlap = 0.25, window = 1)
et = time.time() - start_t
print('Time elapsed: {}'.format(et))

Time elapsed: 0.0014481544494628906


In [75]:
print(signal_segs.shape)

(100747, 12)


In [95]:
def computeTimeFreqRep(signals, freq = 12, overlap = 0.25, window = 1):
    """
    Args:
        signals: EEG segment, shape (number of channels, number of data points)
        freq: sampling frequency in Hz
        overlap: proportion of overlap with previous segment
        window: window size in seconds
    Returns:
        S: time-frequency representation, shape (number of channels, number of data points)
        where number of data points depends on freq, overlap and window.
    """
    n_ch, t_len = signals.shape

    start_time = 0
    end_time = min(t_len, start_time + int(freq * window))
      
    signal_segs = [signals[:,start_time:end_time]]

    while end_time < t_len:        
        offset = int(freq * window * overlap)
        start_time = end_time - offset
        end_time = min(t_len, start_time + int(freq * window))
        curr_seg = signals[:, start_time:end_time]
        #if curr_seg.shape[1] < int(freq * window):
        #    diff = int(freq * window) - curr_seg.shape[1]
        #    curr_seg = np.concatenate((curr_seg, np.zeros((n_ch, diff))), axis=1)
        signal_segs.append(curr_seg)
    
    signal_segs = np.concatenate(signal_segs, axis=1)
    
    return signal_segs

In [101]:
start_t = time.time()
signal_segs2 = computeTimeFreqRep(signals, freq = 96, overlap = 0.75, window = 1)
et = time.time() - start_t
print('Time elapsed: {}'.format(et))

Time elapsed: 0.13117289543151855


In [102]:
print(signal_segs2.shape)

(19, 1208730)


In [103]:
from window_slider import Slider

def sliding_window(signals, freq = 12, overlap = 0.25, window = 1):
    bucket_size = int(freq * window)
    overlap_count = int(freq * window * overlap)

    slider = Slider(bucket_size,overlap_count)
    slider.fit(signals)     
    signal_segs = []
    while True:
        window_data = slider.slide()
        signal_segs.append(window_data)
        if slider.reached_end_of_list(): 
            break
    signal_segs = np.concatenate(signal_segs, axis=1)  
    return signal_segs

In [104]:
start_t = time.time()
signal_segs3 =  sliding_window(signals, freq = 24, overlap = 0.75, window = 1)
et = time.time() - start_t
print('Time elapsed: {}'.format(et))

Time elapsed: 0.32720065116882324


In [105]:
print(signal_segs3.shape)

(19, 1208946)
